In [11]:
import numpy as np 
import pandas as pd 

In [ ]:
# Extract the data of the stock closing prices
df = pd.read_csv(r"../PSEi.csv")
df = df.dropna()
df['Date'] = pd.to_datetime(df['Date'])
stockReturnsTechInd = df.reset_index(drop=True)

In [ ]:
def classifyByDirection(data):
    '''class label for next day direction of data: 1 if increase, 0 if decrease'''
    classArray = []
    for rowIndex in range(len(data)):
        if rowIndex == len(data)-1:
            classLabel = None
        elif data[rowIndex+1] > data[rowIndex]:
            classLabel = 1
        elif data[rowIndex+1] < data[rowIndex]:
            classLabel = 0
        classArray.append(classLabel)
    return classArray
  
stockReturnsTechInd['Class'] = classifyByDirection(stockReturnsTechInd['PSEi'])
stockReturnsTechInd = stockReturnsTechInd.dropna()
stockReturnsTechInd['Class'] = stockReturnsTechInd['Class'].astype('int64')

In [ ]:
def convertToStockReturns(data):
    '''convert the stock closing price to stock returns'''
    srArray = []
    for index in range(len(data)):
        if index == 0:
            srValue = None
        else:
            srValue = np.log(data[index] / data[index-1])
        srArray.append(srValue)
    return srArray

# Create new columns for stock returns of each index
stockReturnsTechInd['PSEi_SR'] = convertToStockReturns(stockReturnsTechInd['PSEi'])
stockReturnsTechInd['PSE SER_SR'] = convertToStockReturns(stockReturnsTechInd['PSE SER'])
stockReturnsTechInd['PSE IND_SR'] = convertToStockReturns(stockReturnsTechInd['PSE IND'])

In [ ]:
def computeSMA(data, ndays: int = 10): 
    '''computes the Simple Moving Average of a period of ndays'''
    SMA = data.rolling(ndays).mean()
    return SMA

stockReturnsTechInd['SMA'] = computeSMA(stockReturnsTechInd['PSEi'])

In [ ]:
def computeWMA(data, ndays: int = 10):
    '''computes the Weighted Moving Average of a period of ndays'''
    ncoeff = list(range(ndays, 0, -1))
    WMA = data.rolling(ndays).apply(lambda x: (ncoeff*x).sum()) / np.sum(ncoeff)
    return WMA

stockReturnsTechInd['WMA'] = computeWMA(stockReturnsTechInd['PSEi'])

In [ ]:
def computeROC(data, ndays: int = 10):
    '''computes the Rate of Change over the period of ndays'''
    ROC = data.diff(ndays-1)
    return ROC

stockReturnsTechInd['ROC'] = computeROC(stockReturnsTechInd['PSEi'])

In [ ]:
def computeStochK(data,ndays: int = 10):
    '''computes Stochastic K % over a period of ndays'''
    lp = data.rolling(ndays).min() # Lowest price of the rolling window 
    hp = data.rolling(ndays).max() # Highest price of the rolling window
    SK = ((data - lp) / (hp - lp)) * 100
    return SK

def computeStochD(data, ndays: int = 10):
    '''Computes the Stochastic D % from the Stochastic K%'''
    SD = data.rolling(ndays).mean()
    return SD

stockReturnsTechInd['Stochastic D'] = computeStochD(stockReturnsTechInd['Stochastic K'])

In [ ]:
def computeRSI(data, ndays: int = 10):
    '''computes for the Relative Strength Index over a period of ndays'''
    closeDifference = data.diff() 
    up = closeDifference.clip(lower=0) # Upward daily price changes 
    down = -1 * closeDifference.clip(upper=0) # Downward daily price changes 
    
    maUp = up.rolling(ndays).mean() # Average gain
    maDown = down.rolling(ndays).mean() # Average loss
    
    ratio = maUp / maDown
    RSI = 100 - (100/(1 + ratio))
    return RSI

stockReturnsTechInd['RSI'] = computeRSI(stockReturnsTechInd['PSEi'])

In [ ]:
def computeEMA(data, ndays: int):
    '''computes for the Exponential Moving Average'''
    EMA = np.zeros(len(data)) 
    beta = 2 / (ndays + 1) # Weight
    EMA[ndays-1] = data.iloc[:ndays].mean() # Compute for first EMA value 
    for index in range(ndays, len(data)): # Compute for rest of EMA values
        EMA[index] = (data[index] * beta) + (1 - beta) * (EMA[index-1])
    return EMA

def computeMACD(data, ndays: int = 10, short_term: int = 12, long_term: int = 26):
    '''computes for the Moving Average Converge Diverge'''
    MACD = np.zeros(len(data))
    alpha = 2 / (ndays + 1) # Weight
    firstDayIndex = long_term + ndays - 1 
    shortEMA = computeEMA(data, short_term) # Short-term EMA default at 12
    longEMA = computeEMA(data, long_term) # Longer-term EMA default at 26
    diffEMA = shortEMA - longEMA # Difference of short-term and longer-term EMA
    MACD[firstDayIndex-1] = diffEMA[(firstDayIndex-ndays):firstDayIndex].mean() # Compute for first MACD value
    for index in range((firstDayIndex), len(data)): # Compute for rest of MACD values
        MACD[index] = (alpha * diffEMA[index]) + ((1-alpha) * MACD[index-1])
    return MACD

stockReturnsTechInd['MACD'] = computeMACD(stockReturnsTechInd['PSEi'])

In [ ]:
stockReturnsTechInd

In [ ]:
# # Save the new file including both stock returns and technical indicators
# stockReturnsTechInd.to_csv('../Data/PSE-SR-TI.csv', index=False)